In [3]:
[hash(v) for v in (0, 1, 2, 3)]

[0, 1, 2, 3]

In [4]:
[hash(v) for v in ("namea", "nameb", "namec", "named")]

[-4154337832941740556,
 7751246281088886980,
 1060499909237031798,
 2686476432335111919]

In [26]:
hash('a'), hash('a') & 7 , hash('b'), hash('b') & 7 

(-9178293282215004519, 1, 8490315224035172066, 2)

In [8]:
hash('z'), hash('z') & 7

(-1517541376421852839, 1)

In [9]:
# -----------------------------------------------------------

In [27]:
"""
A Python dict implementation.
"""

import collections
import pprint

MINSIZE = 8
PERTURB_SHIFT = 5
dummy = "<dummy key>"


In [28]:

class Entry(object):
    """
    A hash table entry.

    Attributes:
       * key - The key for this entry.
       * hash - The has of the key.
       * value - The value associated with the key.
    """

    __slots__ = ("key", "value", "hash")

    def __init__(self):
        self.key = None
        self.value = None
        self.hash = 0

    def __repr__(self):
        return "<Entry: key={0} value={1}>".format(self.key, self.value)


In [34]:

class Dict(object):
    """
    A mapping interface implemented as a hash table.

    Attributes:
        * used - The number of entires used in the table.
        * filled - used + number of entries with a dummy key.
        * table - List of entries; contains the actual dict data.
        * mask - Length of table - 1. Used to fetch values.
    """

    __slots__ = ("filled", "used", "mask", "table")

    
    def __init__(self, arg=None, **kwargs):
        """
        https://github.com/python/cpython/blame/3.7/Objects/dictobject.c#L671
        """
        self.clear()
        self._update(arg, kwargs)

    @classmethod
    def fromkeys(cls, keys, value=0):
        """
        Return a new dictionary from a sequence of keys.
        """
        d = cls()
        for key in keys:
            d[key] = value
        return d

    def clear(self):
        """
        Clear the dictionary of all data.
        """
        self.filled = 0
        self.used = 0
        self.mask = MINSIZE - 1
        self.table = []
        # Initialize the table to a clean slate of entries.
        for i in range(MINSIZE):
            self.table.append(Entry())

    def pop(self, *args):
        """
        Remove and return the value for a key.
        """
        have_default = len(args) == 2
        try:
            v = self[args[0]]
        except KeyError:
            if have_default:
                return args[1]
            raise
        else:
            del self[args[0]]  # call __delitem__
            return v

    def popitem(self):
        """
        Remove and return any key-value pair from the dictionary.
        Note:
            Changed in version 3.7: LIFO order is now guaranteed.
            In prior versions, popitem() would return an arbitrary key/value pair.
            Here return an arbitrary key/value pair.
        """
        if self.used == 0:
            raise KeyError("empty dictionary")
        entry0 = self.table[0]
        entry = entry0
        i = 0
        if entry0.value is None:
            # The first entry in the table's hash is abused to hold the index to
            # the next place to look for a value to pop.
            i = entry0.hash
            # Make the i in range [1, self.mask].
            if i > self.mask or i < 1:
                i = 1
            entry = self.table[i]
            while entry.value is None:
                i += 1
                if i > self.mask:
                    i = 1
                entry = self.table[i]
        res = entry.key, entry.value
        self._del(entry)
        # Set the next place to start.
        entry0.hash = i + 1
        return res

    def setdefault(self, key, default=0):
        """
        If key is in the dictionary, return it. Otherwise, set it to the default
        value.
        """
        val = self._lookup(key).value
        if val is None:
            self[key] = default
            return default
        return val

    def _lookup(self, key):
        """
        Find the entry for a key.
        """
        key_hash = hash(key)
        i = key_hash & self.mask
        entry = self.table[i]
        if entry.key is None or entry is key:
            return entry
        free = None
        if entry.key is dummy:
            free = entry
        elif entry.hash == key_hash and key == entry.key:
            return entry

        perturb = key_hash
        while True:
            i = (i << 2) + i + perturb + 1;
            entry = self.table[i & self.mask]
            if entry.key is None:
                return entry if free is None else free
            if entry.key is key or \
                    (entry.hash == key_hash and key == entry.key):
                return entry
            elif entry.key is dummy and free is None:
                free = dummy
            perturb >>= PERTURB_SHIFT

        assert False, "not reached"

    def _resize(self, minused):
        """
        Resize the dictionary to at least minused.
        """
        print("Calling _resize with: ", minused)
        newsize = MINSIZE
        # Find the smalled value for newsize.
        while newsize <= minused and newsize > 0:
            newsize <<= 1
        oldtable = self.table
        # Create a new table newsize long, for space claiming.
        newtable = []
        while len(newtable) < newsize:
            newtable.append(Entry())
        # Replace the old table.
        self.table = newtable
        self.used = 0
        self.filled = 0
        # Copy the old data into the new table.
        for entry in oldtable:
            if entry.value is not None:
                self._insert_into_clean(entry)
            elif entry.key is dummy:
                entry.key = None
        # TODO: 
        #    Why not assign self.mask before the above for loop,
        #    to let _insert_into_clean use new self.mask ???
        self.mask = newsize - 1
        

    def _insert_into_clean(self, entry):
        """
        Insert an item in a clean dict. This is a helper for resizing.
        """
        i = entry.hash & self.mask
        new_entry = self.table[i]
        perturb = entry.hash
        while new_entry.key is not None:
            i = (i << 2) + i + perturb + 1
            new_entry = self.table[i & self.mask]
            perturb >>= PERTURB_SHIFT
        new_entry.key = entry.key
        new_entry.value = entry.value
        new_entry.hash = entry.hash
        self.used += 1
        self.filled += 1

    def _insert(self, key, value):
        """
        Add a new value to the dictionary or replace an old one.
        """
        entry = self._lookup(key)
        if entry.value is None:
            self.used += 1
            if entry.key is not dummy:
                self.filled += 1
        entry.key = key
        entry.hash = hash(key)
        entry.value = value

    def _del(self, entry):
        """
        Mark an entry as free with the dummy key.
        """
        print("Calling _del with: ", entry)
        entry.key = dummy
        entry.value = None
        self.used -= 1

    def __getitem__(self, key):
        value = self._lookup(key).value
        if value is None:
            # Check if we're a subclass.
            if type(self) is not Dict:
                # Try to call the __missing__ method.
                missing = getattr(self, "__missing__")
                if missing is not None:
                    return missing(key)
            raise KeyError("no such key: {0!r}".format(key))
        return value

    def __setitem__(self, key, what):
        # None is used as a marker for empty entries, so it can't be in a
        # dictionary.
        print("Calling __setitem__ with: ", key, what)
        assert what is not None and key is not None, \
            "key and value must not be None"
        old_used = self.used
        self._insert(key, what)
        # Maybe resize the dict.
        if not (self.used > old_used and
                self.filled*3 >= (self.mask + 1)*2):
            return
        # Large dictionaries (< 5000) are only doubled in size.
        factor = 2 if self.used > 5000 else 4
        self._resize(factor*self.used)

    def __delitem__(self, key):
        entry = self._lookup(key)
        if entry.value is None:
            raise KeyError("no such key: {0!r}".format(key))
        self._del(entry)

    def __contains__(self, key):
        """
        Check if a key is in the dictionary.
        """
        return self._lookup(key).value is not None

    def __eq__(self, other):
        if not isinstance(other, Dict):
            try:
                # Try to coerce the other to a Dict, so we can compare it.
                other = Dict(other)
            except TypeError:
                
                return NotImplemented
        if self.used != other.used:
            # They're not the same size.
            return False
        # Look through the table and compare every entry, breaking out early if
        # we find a difference.
        for entry in self.table:
            if entry.value is not None:
                try:
                    bval = other[entry.key]
                except KeyError:
                    return False
                if not bval == entry.value:
                    return False
        return True

    def __ne__(self, other):
        return not self == other

    def keys(self):
        """
        Return a list of keys in the dictionary.
        """
        return [entry.key for entry in self.table if entry.value is not None]

    def values(self):
        """
        Return a list of values in the dictionary.
        """
        return [entry.value for entry in self.table if entry.value is not None]

    def items(self):
        """
        Return a list of key-value pairs.
        """
        print("Calling items ...")
        return [(entry.key, entry.value) for entry in self.table
                if entry.value is not None]

    def __iter__(self):
        print("Calling __iter__ ...")
        return DictKeysIterator(self)

    def itervalues(self):
        """
        Return an iterator over the values in the dictionary.
        """
        print("Calling itervalues ...")
        return DictValuesIterator(self)

    def iterkeys(self):
        """
        Return an iterator over the keys in the dictionary.
        """
        print("Calling iterkeys ...")
        return DictKeysIterator(self)

    def iteritems(self):
        """
        Return an iterator over key-value pairs.
        """
        print("Calling iteritems ...")
        return DictItemsIterator(self)

    def _merge(self, mapping):
        """
        Update the dictionary from a mapping.
        """
        for key in mapping.keys():
            self[key] = mapping[key]

    def _from_sequence(self, seq):
        for double in seq:
            if len(double) != 2:
                raise ValueError("{0!r} doesn't have a length of 2".format(
                        double))
            self[double[0]] = double[1]

    def _update(self, arg, kwargs):
        print("Calling _update with: ", arg, kwargs)
        if arg:
            # arg is also a instance of Mapping.
            if isinstance(arg, collections.Mapping):
                self._merge(arg)
            else:
                self._from_sequence(arg)
        # Handle when calling with self._update(one=1, two=2, three=3)
        if kwargs:
            self._merge(kwargs)

    def update(self, arg=None, **kwargs):
        """
        Update the dictionary from a mapping or sequence containing key-value
        pairs. Any existing values are overwritten.
        """
        self._update(arg, kwargs)

    def get(self, key, default=0):
        """
        Return the value for key if it exists otherwise the default.
        """
        try:
            return self[key]
        except KeyError:
            return default

    def __len__(self):
        return self.used

    def __repr__(self):
        r = ["{0!r} : {1!r}".format(k, v) for k, v in self.iteritems()]
        return "Dict({" + ", ".join(r) + "})"


collections.Mapping.register(Dict) 
# https://docs.python.org/3/library/abc.html#abc.ABCMeta.register


class DictIterator(object):

    def __init__(self, d):
        self.d = d
        self.used = self.d.used
        self.len = self.d.used
        self.pos = 0

    def __iter__(self):
        return self

    def next(self):
        # Check if the dictionary has been mutated under us.
        if self.used != self.d.used:
            # Make this state permanent.
            self.used = -1
            raise RuntimeError("dictionary size changed during interation")
        i = self.pos
        while i <= self.d.mask and self.d.table[i].value is None:
            i += 1
        self.pos = i + 1
        if i > self.d.mask:
            # We're done.
            raise StopIteration
        self.len -= 1
        return self._extract(self.d.table[i])

    __next__ = next

    def _extract(self, entry):
        return getattr(entry, self.kind)

    def __len__(self):
        return self.len

class DictKeysIterator(DictIterator):
    kind = "key"

class DictValuesIterator(DictIterator):
    kind = "value"

class DictItemsIterator(DictIterator):

    def _extract(self, entry):
        return entry.key, entry.value

In [30]:
dir(Dict)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_del',
 '_from_sequence',
 '_insert',
 '_insert_into_clean',
 '_lookup',
 '_merge',
 '_resize',
 '_update',
 'clear',
 'filled',
 'fromkeys',
 'get',
 'items',
 'iteritems',
 'iterkeys',
 'itervalues',
 'keys',
 'mask',
 'pop',
 'popitem',
 'setdefault',
 'table',
 'update',
 'used',
 'values']

In [36]:
# https://docs.python.org/3/library/stdtypes.html#mapping-types-dict

# Init
a = Dict(one=1, two=2, three=3)
b = Dict({'one': 1, 'two': 2, 'three': 3})
c = Dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = Dict([('two', 2), ('one', 1), ('three', 3)])
e = Dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e
a = Dict([('two', 2), ('one', 1), ('three', 3)], one=1, two=2, three=3)


Calling _update with:  None {'one': 1, 'two': 2, 'three': 3}
Calling __setitem__ with:  one 1
Calling __setitem__ with:  two 2
Calling __setitem__ with:  three 3
Calling _update with:  {'one': 1, 'two': 2, 'three': 3} {}
Calling __setitem__ with:  one 1
Calling __setitem__ with:  two 2
Calling __setitem__ with:  three 3
Calling _update with:  <zip object at 0x7f3358083048> {}
Calling __setitem__ with:  one 1
Calling __setitem__ with:  two 2
Calling __setitem__ with:  three 3
Calling _update with:  [('two', 2), ('one', 1), ('three', 3)] {}
Calling __setitem__ with:  two 2
Calling __setitem__ with:  one 1
Calling __setitem__ with:  three 3
Calling _update with:  {'three': 3, 'one': 1, 'two': 2} {}
Calling __setitem__ with:  three 3
Calling __setitem__ with:  one 1
Calling __setitem__ with:  two 2
Calling _update with:  [('two', 2), ('one', 1), ('three', 3)] {'one': 1, 'two': 2, 'three': 3}
Calling __setitem__ with:  two 2
Calling __setitem__ with:  one 1
Calling __setitem__ with:  three 

In [32]:
# Insert & Get 

f = Dict()
f['two'] = 2
f.setdefault('three', 3)
print(f)
f['two'], f.get('two')

Calling _update with:  None {}
Calling __setitem__ with:  two 2
Calling __setitem__ with:  three 3
Calling iteritems ...
Dict({'two' : 2, 'three' : 3})


(2, 2)

In [33]:
# Delete
g = Dict(one=1, two=2, three=3)
t1 = g.pop('zero', 100)
t2 = g.pop('two')
t3 = g.popitem()
t4 = g.popitem()
print(g)
g['Five'] = 5
g.clear()

t1, t2, t3, t4, g

Calling _update with:  None {'one': 1, 'two': 2, 'three': 3}
Calling __setitem__ with:  one 1
Calling __setitem__ with:  two 2
Calling __setitem__ with:  three 3
Calling _del with:  <Entry: key=two value=2>
Calling _del with:  <Entry: key=three value=3>
Calling _del with:  <Entry: key=one value=1>
Calling iteritems ...
Dict({})
Calling __setitem__ with:  Five 5
Calling iteritems ...


(100, 2, ('three', 3), ('one', 1), Dict({}))

In [17]:
# Update & Dict resizing
d = Dict()
#print(d, type(d))
d['key1'] = 'value1'
pprint.pprint(d.table)
print(d, type(d), d.used, d.mask+1)
print("----------------------------")
d.update(zip(range(10), range(10)))
print(d, type(d), d.used, d.mask+1)


Calling _update with:  None {}
Calling __setitem__ with:  key1 value1
[<Entry: key=None value=None>,
 <Entry: key=None value=None>,
 <Entry: key=None value=None>,
 <Entry: key=key1 value=value1>,
 <Entry: key=None value=None>,
 <Entry: key=None value=None>,
 <Entry: key=None value=None>,
 <Entry: key=None value=None>]
Calling iteritems ...
Dict({'key1' : 'value1'}) <class '__main__.Dict'> 1 8
----------------------------
Calling _update with:  <zip object at 0x7f335f478448> {}
Calling __setitem__ with:  0 0
Calling __setitem__ with:  1 1
Calling __setitem__ with:  2 2
Calling __setitem__ with:  3 3
Calling __setitem__ with:  4 4
Calling _resize with:  24
Calling __setitem__ with:  5 5
Calling __setitem__ with:  6 6
Calling __setitem__ with:  7 7
Calling __setitem__ with:  8 8
Calling __setitem__ with:  9 9
Calling iteritems ...
Dict({0 : 0, 1 : 1, 2 : 2, 'key1' : 'value1', 4 : 4, 5 : 5, 3 : 3, 7 : 7, 8 : 8, 9 : 9, 6 : 6}) <class '__main__.Dict'> 11 32


In [18]:
# Iterate
d = Dict(zip(['One', 'Two', 'Thr', 'For'], range(4)))
print("----------------------------")

for k in d:
    print(k, d[k])
print("----------------------------")

for k, v in d.items():
    print(k, v)
print("----------------------------")

print(d.keys())
print(d.values())
print("----------------------------")

dv = d.itervalues()
print(dv, type(dv))
for v in dv:
    print(v)
print("----------------------------")

Calling _update with:  <zip object at 0x7f3358098288> {}
Calling __setitem__ with:  One 0
Calling __setitem__ with:  Two 1
Calling __setitem__ with:  Thr 2
Calling __setitem__ with:  For 3
----------------------------
Calling __iter__ ...
For 3
Two 1
One 0
Thr 2
----------------------------
Calling items ...
For 3
Two 1
One 0
Thr 2
----------------------------
['For', 'Two', 'One', 'Thr']
[3, 1, 0, 2]
----------------------------
Calling itervalues ...
<__main__.DictValuesIterator object at 0x7f33580e49b0> <class '__main__.DictValuesIterator'>
3
1
0
2
----------------------------


In [160]:
#=======================================================================================

In [159]:
 # class __dict__

In [24]:
class C(object):
    x = 4

c = C()
c.y = 5
c.__dict__, type(c.__dict__)


({'y': 5}, dict)

In [25]:
c.x, C.x

(4, 4)

In [20]:
C.__dict__, type(C.__dict__)

(mappingproxy({'__module__': '__main__',
               'x': 4,
               '__dict__': <attribute '__dict__' of 'C' objects>,
               '__weakref__': <attribute '__weakref__' of 'C' objects>,
               '__doc__': None}),
 mappingproxy)

In [21]:
# https://github.com/python/cpython/blob/3.7/Lib/_collections_abc.py#L691
isinstance(C.__dict__  , collections.Mapping)

True

In [22]:
type(type.__dict__)

mappingproxy

In [23]:
type.__dict__

mappingproxy({'__repr__': <slot wrapper '__repr__' of 'type' objects>,
              '__call__': <slot wrapper '__call__' of 'type' objects>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'type' objects>,
              '__setattr__': <slot wrapper '__setattr__' of 'type' objects>,
              '__delattr__': <slot wrapper '__delattr__' of 'type' objects>,
              '__init__': <slot wrapper '__init__' of 'type' objects>,
              '__new__': <function type.__new__(*args, **kwargs)>,
              'mro': <method 'mro' of 'type' objects>,
              '__subclasses__': <method '__subclasses__' of 'type' objects>,
              '__prepare__': <method '__prepare__' of 'type' objects>,
              '__instancecheck__': <method '__instancecheck__' of 'type' objects>,
              '__subclasscheck__': <method '__subclasscheck__' of 'type' objects>,
              '__dir__': <method '__dir__' of 'type' objects>,
              '__sizeof__': <method '__sizeof__